In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import gmaps
from census import Census

In [2]:
police_killings_df = pd.read_csv('police_killings.csv')
education_df = pd.read_csv('education_census_bureau_totals.csv')

In [3]:
del police_killings_df['URL of image of victim']
police_killings_df.head()

,Victim's name,Victim's age,Victim's gender,Victim's race,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,County,...,Symptoms of mental illness?,Unarmed,Alleged Weapon (Source: WaPo),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Off-Duty Killing?,Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx ),ID
0,Eric M. Tellez,28,Male,White,31-12-19,Broad St.,Globe,AZ,85501.0,Gila,...,No,Allegedly Armed,knife,other,not fleeing,no,5332.0,NaN,Rural,7664
1,Name withheld by police,NaN,Male,Unknown race,31-12-19,7239-7411 I-40,Memphis,AR,38103.0,Crittenden,...,No,Unclear,unclear,other,NaN,NaN,NaN,NaN,Urban,7665
2,Terry Hudson,57,Male,Black,31-12-19,3600 N 24th St,Omaha,NE,68110.0,Douglas,...,No,Allegedly Armed,gun,attack,not fleeing,no,5359.0,NaN,Urban,7661
3,Malik Williams,23,Male,Black,31-12-19,30800 14th Avenue South,Federal Way,WA,98003.0,King,...,No,Allegedly Armed,gun,attack,not fleeing,no,5358.0,NaN,Suburban,7662
4,Frederick Perkins,37,Male,Black,31-12-19,17057 N Outer 40 Rd,Chesterfield,MO,63005.0,St. Louis,...,No,Vehicle,vehicle,attack,car,no,5333.0,NaN,Suburban,7667


In [4]:
police_killings_df.columns

Index(['Victim's name', 'Victim's age', 'Victim's gender', 'Victim's race',
       'Date of Incident (month/day/year)', 'Street Address of Incident',
       'City', 'State', 'Zipcode', 'County', 'Agency responsible for death',
       'Cause of death',
       'A brief description of the circumstances surrounding the death',
       'Official disposition of death (justified or other)',
       'Criminal Charges?',
       'Link to news article or photo of official document',
       'Symptoms of mental illness?', 'Unarmed',
       'Alleged Weapon (Source: WaPo)', 'Alleged Threat Level (Source: WaPo)',
       'Fleeing (Source: WaPo)', 'Body Camera (Source: WaPo)',
       'WaPo ID (If included in WaPo database)', 'Off-Duty Killing?',
       'Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx )',
       'ID'],
      dtype='object')

In [5]:
police_killings_df.rename(columns = {
    "Victim's name" : 'Victim_Name',
    "Victim's age" : 'Victim_age',
    "Victim's gender" : 'Victim_gender',
    "Victim's race" : 'Victim_race',
    'Date of Incident (month/day/year)' : 'Incident_Date',
    'Street Address of Incident' : 'Incident_Address_Street',
    'Agency responsible for death' : 'Agency_Responsible_Death',
    'Cause of death' : 'Death_Cause',
    'A brief description of the circumstances surrounding the death' : 'Death_Circumstances',
    'Official disposition of death (justified or other)' : 'Official_Disposition_Death',
    'Criminal Charges?' : 'Criminal_Charges?',
    'Link to news article or photo of official document' : 'Link_Article_Photo',
    'Symptoms of mental illness?' : 'Symptoms_Mental_Illness?',
    'Alleged Weapon (Source: WaPo)' : 'Alleged_Weapon',
    'Alleged Threat Level (Source: WaPo)' : 'Alleged_Threat_Level',
    'Fleeing (Source: WaPo)' : 'Fleeing',
    'Body Camera (Source: WaPo)' : 'Body_Camera',
    'WaPo ID (If included in WaPo database)' : 'WaPo_ID',
    'Off-Duty Killing?' : 'Off-Duty_Killing?',
    'Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx )' : 'Geography',
})

,Victim_Name,Victim_age,Victim_gender,Victim_race,Incident_Date,Incident_Address_Street,City,State,Zipcode,County,...,Symptoms_Mental_Illness?,Unarmed,Alleged_Weapon,Alleged_Threat_Level,Fleeing,Body_Camera,WaPo_ID,Off-Duty_Killing?,Geography,ID
0,Eric M. Tellez,28,Male,White,31-12-19,Broad St.,Globe,AZ,85501.0,Gila,...,No,Allegedly Armed,knife,other,not fleeing,no,5332.0,NaN,Rural,7664
1,Name withheld by police,NaN,Male,Unknown race,31-12-19,7239-7411 I-40,Memphis,AR,38103.0,Crittenden,...,No,Unclear,unclear,other,NaN,NaN,NaN,NaN,Urban,7665
2,Terry Hudson,57,Male,Black,31-12-19,3600 N 24th St,Omaha,NE,68110.0,Douglas,...,No,Allegedly Armed,gun,attack,not fleeing,no,5359.0,NaN,Urban,7661
3,Malik Williams,23,Male,Black,31-12-19,30800 14th Avenue South,Federal Way,WA,98003.0,King,...,No,Allegedly Armed,gun,attack,not fleeing,no,5358.0,NaN,Suburban,7662
4,Frederick Perkins,37,Male,Black,31-12-19,17057 N Outer 40 Rd,Chesterfield,MO,63005.0,St. Louis,...,No,Vehicle,vehicle,attack,car,no,5333.0,NaN,Suburban,7667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7658,Mark Chavez,49,Male,Hispanic,01-01-13,912 Loma Linda Ave.,Farmington,NM,87401.0,San Juan,...,No,Allegedly Armed,blunt weapon,NaN,NaN,NaN,NaN,NaN,Urban,1
7659,Tyree Bell,31,Male,Black,01-01-13,3727 N. 42nd St.,Omaha,NE,68111.0,Douglas,...,Yes,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN,Urban,2
7660,Christopher Tavares,21,Male,Hispanic,01-01-13,Highway 50 and North Elizabeth Street,Pueblo,CO,81008.0,Pueblo,...,No,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN,Suburban,4
7661,Andrew L. Closson,21,Male,White,01-01-13,U.S. Highway 53,Gordon,WI,54838.0,Douglas,...,Drug or alcohol use,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN,Rural,5


In [6]:
education_df.columns

Index(['id', 'Geographic Area Name', 'Total_population_18_24',
       'Total_18_24_lt_high_school', 'Total_18_24_high_school_diploma',
       'Total_18_24_associate_degree', 'Total_18_24_bachelor',
       'Total_population_25_and_over', 'Total_population_25_lt_9th',
       'Total_25_9_12_no_diploma', 'Total_25_high_School',
       'Total_population_25_college_no_degree',
       'Total_population_25_associate_degree', 'Total_population_25 _bachelor',
       'Total_population_25_graduate', 'Total_25_over_High_school',
       'Total_population_25years_and_over_Bachelor', 'Total_population_25_34',
       'Total_25_34_High_school', 'Total_25_34_Bachelor_higher',
       'Total_population_35_44', 'Total_35_44_high_school_graduate',
       'Total_35_44_bachelor_degree', 'Total_population_45_64_years',
       'Total_45_64_high_school', 'Total_45_64_bachelor_degree',
       'Total_population_65_years_over', 'Total_65_high_school',
       'Total_65_bachelor_degree'],
      dtype='object')

In [7]:
education_df['Geographic Area Name']

def get_city(address):
    return address.split(",")[0].strip(" ")

def get_state(address):
    return address.split(",")[1].split(" ")[1]

education_df['City'] = education_df['Geographic Area Name'].apply(lambda x: f"{get_city(x)}")
education_df['State'] = education_df['Geographic Area Name'].apply(lambda x: f"{get_state(x)}")
education_df.head()

,id,Geographic Area Name,Total_population_18_24,Total_18_24_lt_high_school,Total_18_24_high_school_diploma,Total_18_24_associate_degree,Total_18_24_bachelor,Total_population_25_and_over,Total_population_25_lt_9th,Total_25_9_12_no_diploma,...,Total_35_44_high_school_graduate,Total_35_44_bachelor_degree,Total_population_45_64_years,Total_45_64_high_school,Total_45_64_bachelor_degree,Total_population_65_years_over,Total_65_high_school,Total_65_bachelor_degree,City,State
0,310M400US10100,"Aberdeen, SD Micro Area",4294,652,1135,2049,458,28245,1008,1000,...,4406,1589,10764,10306,3114,7226,6282,1583,Aberdeen,SD
1,310M400US10140,"Aberdeen, WA Micro Area",5171,803,2038,2103,227,51839,1969,3792,...,7196,1399,20515,18297,3388,14607,13056,2425,Aberdeen,WA
2,310M400US10180,"Abilene, TX Metro Area",22201,2699,7417,10364,1721,107411,5195,8794,...,17141,4283,37959,32936,8604,24874,20693,5848,Abilene,TX
3,310M400US10220,"Ada, OK Micro Area",4426,561,1248,2353,264,24796,655,2156,...,3959,1375,8910,8021,2441,6148,5192,1636,Ada,OK
4,310M400US10260,"Adjuntas, PR Micro Area",1719,302,331,927,159,12554,3082,1452,...,1558,463,4954,3261,830,3230,1238,334,Adjuntas,PR


In [8]:
# Find NAN
nan_df = police_killings_df[police_killings_df.isna().any(axis=1)]
nan_df

,Victim's name,Victim's age,Victim's gender,Victim's race,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,County,...,Symptoms of mental illness?,Unarmed,Alleged Weapon (Source: WaPo),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Off-Duty Killing?,Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx ),ID
0,Eric M. Tellez,28,Male,White,31-12-19,Broad St.,Globe,AZ,85501.0,Gila,...,No,Allegedly Armed,knife,other,not fleeing,no,5332.0,NaN,Rural,7664
1,Name withheld by police,NaN,Male,Unknown race,31-12-19,7239-7411 I-40,Memphis,AR,38103.0,Crittenden,...,No,Unclear,unclear,other,NaN,NaN,NaN,NaN,Urban,7665
2,Terry Hudson,57,Male,Black,31-12-19,3600 N 24th St,Omaha,NE,68110.0,Douglas,...,No,Allegedly Armed,gun,attack,not fleeing,no,5359.0,NaN,Urban,7661
3,Malik Williams,23,Male,Black,31-12-19,30800 14th Avenue South,Federal Way,WA,98003.0,King,...,No,Allegedly Armed,gun,attack,not fleeing,no,5358.0,NaN,Suburban,7662
4,Frederick Perkins,37,Male,Black,31-12-19,17057 N Outer 40 Rd,Chesterfield,MO,63005.0,St. Louis,...,No,Vehicle,vehicle,attack,car,no,5333.0,NaN,Suburban,7667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7658,Mark Chavez,49,Male,Hispanic,01-01-13,912 Loma Linda Ave.,Farmington,NM,87401.0,San Juan,...,No,Allegedly Armed,blunt weapon,NaN,NaN,NaN,NaN,NaN,Urban,1
7659,Tyree Bell,31,Male,Black,01-01-13,3727 N. 42nd St.,Omaha,NE,68111.0,Douglas,...,Yes,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN,Urban,2
7660,Christopher Tavares,21,Male,Hispanic,01-01-13,Highway 50 and North Elizabeth Street,Pueblo,CO,81008.0,Pueblo,...,No,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN,Suburban,4
7661,Andrew L. Closson,21,Male,White,01-01-13,U.S. Highway 53,Gordon,WI,54838.0,Douglas,...,Drug or alcohol use,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN,Rural,5


In [9]:
police_killings_df = police_killings_df.dropna(how='all')
police_killings_df

,Victim's name,Victim's age,Victim's gender,Victim's race,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,County,...,Symptoms of mental illness?,Unarmed,Alleged Weapon (Source: WaPo),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Off-Duty Killing?,Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx ),ID
0,Eric M. Tellez,28,Male,White,31-12-19,Broad St.,Globe,AZ,85501.0,Gila,...,No,Allegedly Armed,knife,other,not fleeing,no,5332.0,NaN,Rural,7664
1,Name withheld by police,NaN,Male,Unknown race,31-12-19,7239-7411 I-40,Memphis,AR,38103.0,Crittenden,...,No,Unclear,unclear,other,NaN,NaN,NaN,NaN,Urban,7665
2,Terry Hudson,57,Male,Black,31-12-19,3600 N 24th St,Omaha,NE,68110.0,Douglas,...,No,Allegedly Armed,gun,attack,not fleeing,no,5359.0,NaN,Urban,7661
3,Malik Williams,23,Male,Black,31-12-19,30800 14th Avenue South,Federal Way,WA,98003.0,King,...,No,Allegedly Armed,gun,attack,not fleeing,no,5358.0,NaN,Suburban,7662
4,Frederick Perkins,37,Male,Black,31-12-19,17057 N Outer 40 Rd,Chesterfield,MO,63005.0,St. Louis,...,No,Vehicle,vehicle,attack,car,no,5333.0,NaN,Suburban,7667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7658,Mark Chavez,49,Male,Hispanic,01-01-13,912 Loma Linda Ave.,Farmington,NM,87401.0,San Juan,...,No,Allegedly Armed,blunt weapon,NaN,NaN,NaN,NaN,NaN,Urban,1
7659,Tyree Bell,31,Male,Black,01-01-13,3727 N. 42nd St.,Omaha,NE,68111.0,Douglas,...,Yes,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN,Urban,2
7660,Christopher Tavares,21,Male,Hispanic,01-01-13,Highway 50 and North Elizabeth Street,Pueblo,CO,81008.0,Pueblo,...,No,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN,Suburban,4
7661,Andrew L. Closson,21,Male,White,01-01-13,U.S. Highway 53,Gordon,WI,54838.0,Douglas,...,Drug or alcohol use,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN,Rural,5


In [10]:
police_killings_df.dtypes

Victim's name                                                                                                                                                                  object
Victim's age                                                                                                                                                                   object
Victim's gender                                                                                                                                                                object
Victim's race                                                                                                                                                                  object
Date of Incident (month/day/year)                                                                                                                                              object
Street Address of Incident                                                                

In [11]:
police_st_grp = police_killings_df.groupby(['State'])

In [12]:
mo_killngs_df = pd.DataFrame(police_st_grp.get_group('MO')).reset_index()
del mo_killngs_df['index']
mo_killngs_df

,Victim's name,Victim's age,Victim's gender,Victim's race,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,County,...,Symptoms of mental illness?,Unarmed,Alleged Weapon (Source: WaPo),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Off-Duty Killing?,Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx ),ID
0,Frederick Perkins,37,Male,Black,31-12-19,17057 N Outer 40 Rd,Chesterfield,MO,63005.0,St. Louis,...,No,Vehicle,vehicle,attack,car,no,5333.0,NaN,Suburban,7667
1,Cody McCaulou,27,Male,White,30-12-19,706 Hickman Ave,Fulton,MO,65251.0,Callaway,...,No,Vehicle,vehicle,attack,car,no,5404.0,NaN,Rural,7659
2,David A. White,56,Male,Unknown race,24-12-19,521 Latoka Ct.,Springfield,MO,65807.0,Greene,...,No,Allegedly Armed,gun,other,not fleeing,no,5399.0,NaN,Urban,7639
3,Cortez Bufford,24,Male,Black,12-12-19,Bates St and Virginia Ave,St. Louis,MO,63111.0,St. Louis City,...,No,Allegedly Armed,gun,other,Foot,No,5280.0,NaN,Urban,7594
4,Cameron Lamb,26,Male,Black,03-12-19,4100 College Ave,Kansas City,MO,64130.0,Jackson,...,No,Allegedly Armed,gun,other,Not fleeing,No,5250.0,NaN,Urban,7568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Thomas Schroeder,50,Male,White,28-01-13,Interstate 29,Craig,MO,64437.0,Holt,...,Drug or alcohol use,Unarmed,unarmed,NaN,NaN,NaN,NaN,NaN,Rural,97
193,Donovan Thomas,22,Male,Black,19-01-13,3524 N. Kingshighway Blvd.,St. Louis,MO,63115.0,St. Louis,...,No,Unarmed,toy,NaN,NaN,NaN,NaN,Off-Duty,Urban,76
194,Todd S. Weber,38,Male,White,15-01-13,Front St & Interstate 435,Kansas City,MO,64120.0,Jackson,...,No,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN,Suburban,57
195,Gilbert S. Owens,NaN,Male,Unknown race,13-01-13,North East Street,Senath,MO,63876.0,Dunklin,...,Unknown,Allegedly Armed,knife,NaN,NaN,NaN,NaN,NaN,Rural,50


In [13]:
ed_st_grp = education_df.groupby(['State'])

In [14]:
missouri_ed_df = pd.DataFrame(ed_st_grp.get_group('MO')).reset_index()
del missouri_ed_df['index']
missouri_ed_df

,id,Geographic Area Name,Total_population_18_24,Total_18_24_lt_high_school,Total_18_24_high_school_diploma,Total_18_24_associate_degree,Total_18_24_bachelor,Total_population_25_and_over,Total_population_25_lt_9th,Total_25_9_12_no_diploma,...,Total_35_44_high_school_graduate,Total_35_44_bachelor_degree,Total_population_45_64_years,Total_45_64_high_school,Total_45_64_bachelor_degree,Total_population_65_years_over,Total_65_high_school,Total_65_bachelor_degree,City,State
0,310M400US14700,"Branson, MO Micro Area",7406,1149,2531,3368,358,61873,1615,5905,...,7543,1898,23623,20963,4106,20720,18079,3788,Branson,MO
1,310M400US17860,"Columbia, MO Metro Area",35104,1465,5056,23232,5351,105495,1826,4335,...,19313,9990,37930,35481,16756,20163,18625,7909,Columbia,MO
2,310M400US22100,"Farmington, MO Micro Area",5618,993,2393,2116,116,46667,1891,5374,...,7951,1579,17707,15039,2577,10506,8191,1365,Farmington,MO
3,310M400US22780,"Fort Leonard Wood, MO Micro Area",11946,700,5430,5475,341,29000,805,1687,...,5336,1441,9507,8597,2041,4373,3586,825,Fort Leonard Wood,MO
4,310M400US25300,"Hannibal, MO Micro Area",3301,408,1200,1435,258,26530,1003,2054,...,4141,936,10627,9557,1953,7006,5722,993,Hannibal,MO
5,310M400US27620,"Jefferson City, MO Metro Area",14034,2797,4047,6185,1005,103086,3792,7370,...,16947,5417,40820,37001,11419,23500,20047,5384,Jefferson City,MO
6,310M400US27900,"Joplin, MO Metro Area",16186,1942,6162,7129,953,117072,5066,10243,...,18073,5497,44230,38911,9632,27815,23883,5084,Joplin,MO
7,310M400US28380,"Kennett, MO Micro Area",2501,579,1079,823,20,20069,2097,2822,...,3013,642,7832,6028,858,5460,3485,632,Kennett,MO
8,310M400US28860,"Kirksville, MO Micro Area",7640,442,1369,5037,792,16460,650,1038,...,2303,842,6295,5711,1542,4466,3756,1018,Kirksville,MO
9,310M400US30060,"Lebanon, MO Micro Area",2672,472,953,870,377,24145,1009,2955,...,3428,511,9618,8264,1599,6149,4780,760,Lebanon,MO
